In [ ]:
!pip install tensorflow==2.3.0
!pip install gym
!pip install keras
!pip install keras-rl2

  Using cached tensorflow-2.3.0-cp36-cp36m-win_amd64.whl (342.5 MB)
  Using cached numpy-1.18.5-cp36-cp36m-win_amd64.whl (12.7 MB)
  Using cached tensorflow_estimator-2.3.0-py2.py3-none-any.whl (459 kB)
  Using cached h5py-2.10.0-cp36-cp36m-win_amd64.whl (2.4 MB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
  Using cached scipy-1.4.1-cp36-cp36m-win_amd64.whl (30.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'c:\\users\\mink\\anaconda3\\envs\\openai\\lib\\site-packages\\~-mpy\\.libs\\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.



  Using cached numpy-1.19.5-cp36-cp36m-win_amd64.whl (13.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5


In [ ]:
from gym import Env
from gym.spaces import Discrete, Box
import numpy as np
import random
import gym

편의점 방문 빈도 수에 따른 문 여닫는 이벤트를 푸아송 분포로 나타낸 식
O(t) = Poisson(λ(t))
이제 주어진 시간대별 평균 문 여닫는 이벤트의 수를 나타내는 λ(t)를 사용하여 파이썬 코드로 구현해 보았음.
시간대별로 냉장고 문 여닫는 이벤트 발생 횟수의 평균인 λ(t)를 설정
이 값들은 임의로 설정된 것이며 실제 사용 상황에 맞추어 조정이 필요

In [ ]:
# Example of time-varying lambda (average number of door openings per hour)
lambda_t = np.array([1, 0.5, 0.2, 0.1, 0.1, 0.2, 1, 2, 3, 2, 1.5, 1, 1, 1, 1.5, 2, 3, 4, 5, 4, 3, 2, 1, 0.5])

이 함수는 시간대별 냉장고 문 여닫는 이벤트를 푸아송 분포를 사용하여 계산. 함수는 람다 값들의 리스트인 lambda_t와 냉장고의 길이(초)인 refri_length를 인수로 받음.
 함수는 시간대별로 푸아송 분포에서 샘플링된 이벤트의 수를 저장하는 리스트를 반환.

In [ ]:
def door_opening_event_hourly(lambda_t, t_max):
    event_count = np.random.poisson(lambda_t, (t_max // 3600) + 1)
    hourly_event_count = np.repeat(event_count, 3600)[:t_max]
    return hourly_event_count

처음의 CustomActionSpace 클래스는 custom action class이다. 현재는 기존의 방식인 Discrete를 이용하여 구현했지만 이것이 의도한대로 작동하지 않을 경우를 위해 미리 작성해 둔 class이다.
이는 내가 원하는 임의의 정수들을 action으로 지정하여 그것을 sampling하여 반환하도록 할 수 있고 이 action class 또한 step() 함수의 action 인자로 사용할 수 있다. contain함수는 주어진 action이 Custom action space에 속하는지 여부를 반환하는 함수이다.

현재 구현되어 있는 action_space는 mapping을 이용하여 구현해 두었다. 알아본 결과 Discrete는 연속하는 정수만을 담을 수 있고, 내가 원하는 임의의 불연속적인 정수를 담는 것이 불가능하다. 그래서 일단 Discrete를 이용하여 4개의 action_space를 구현하고 그 action space에 내가 원하는 정수(우리는 온도를 의미)를 mapping하여 action_space를 내가 원하는 value로 변환하도록 구현되었다. 이 방법을 사용하려면 action 대신 action_to_value로 바꾸어 호출하면 된다.
-> 나중에 모든 코드가 구현되고 나서 확인이 필요할 듯 하다.

In [ ]:
class CustomActionSpace(gym.Space):
    def __init__(self):
        self.action_space = Discrete(4)
        self.action_values = [-5, 0, 5, 10]

    def sample(self):
        return self.action_values[self.action_space.sample()]
    
    def contains(self, x):
        return x in self.action_values

RefriEnv 클래스의 reset 메소드를 수정함. 여기서 door_opening_event_hourly 함수를 호출하여 시간대별 문 여닫는 이벤트 수를 계산하고, 이를 self.hourly_event_count 속성에 저장. 이 속성은 환경이 초기화될 때마다 업데이트됨.

In [ ]:
class RefriEnv(Env):
    def __init__(self):
        #action space
        custom_action_space = Discrete(4)
        self.action_space = custom_action_space
        # The real values are that mapping the action value
        # 0 to 3 is mapping the -5,0,5,10
        self.action_to_value = {0: -5, 1: 0, 2: 5, 3: 10}
        # Temperature array
        self.observation_space = Box(low=np.array([-50]), high=np.array([50]))
        # Set start temp
        self.state = 0 + random.randint(-3,3)
        # Set refrigerator length
        self.refri_length = 60

         # O(t)에 관한 코드
        self.hourly_event_count = door_opening_event_hourly(lambda_t, self.refri_length)

    #transform the action to value
    def map_action_to_value(self, action):
        return self.action_to_value[action]
        
    #transform the value to action
    def map_value_to_action(self, value):
        for action, val in self.action_to_value.items():
            if val == value:
                return action
        raise ValueError("Invalid value: {}".format(value))
    
    #RefriEnv 클래스의 step 메소드를 수정했음. 
    #이 메소드에서 시간대별 문 여닫는 이벤트 수를 고려하여 냉장고의 온도가 업데이트되는 방식을 변경함. 
    #먼저, 현재 시간대에서 발생하는 이벤트 수를 계산하고, 이 값을 O(t)로 사용
    def step(self, action):
        # 냉장고의 현재 시간대 계산 //(O(t)에 관한 코드)
        #self.refri_length는 냉장고의 에피소드 길이가 몇 초 남았는지를 나타내는 변수
        #에피소드가 진행될수록 self.refri_length 값이 감소하며, 0이 되면 에피소드가 종료
        current_hour = (self.refri_length - 1) // 3600

        # 현재 시간대에 대한 문 여닫는 이벤트 수//(O(t)에 관한 코드)
        #self.hourly_event_count는 시간대별 문 여닫는 이벤트 수를 저장하는 리스트
        #current_hour 인덱스를 사용하여 현재 시간대에 해당하는 문 여닫는 이벤트 수를 가져옵니다. 이 값을 변수 O에 저장
        O = self.hourly_event_count[current_hour]

        # Apply action
        if(self.state>action):
            self.state += (P_cool*E-U*A*(self.state-T_ext)+O*h*(T_ext-self.state))/(m*c)
        else:
            self.state += (-U*A*(self.state-T_ext)+O*h*(T_ext-self.state))/(m*c)
        
        # Reduce refrigerator length by 1 second
        self.refri_length -= 1 
        
        # Calculate reward
        if self.state>action: 
            reward = -10 
        else: 
            reward= -(self.state-action)
        
        # Check if episode is done
        if self.refri_length <= 0: 
            done = True
        else:
            done = False
        
        # Apply temperature noise
        #self.state += random.randint(-1,1)
        # Set placeholder for info
        info = {}
        
        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass
    
    def reset(self):
        # Reset refrigerator temperature
        self.state = 0 + random.randint(-3,3)
        # Reset refrigerator time
        self.refri_length = 60 
        return self.state

In [ ]:
env = RefriEnv()

In [ ]:
env.observation_space.sample()

array([5.2091026], dtype=float32)

In [ ]:
episodes = 10
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        #env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))

Episode:1 Score:-58
Episode:2 Score:-42
Episode:3 Score:16
Episode:4 Score:-58
Episode:5 Score:-20
Episode:6 Score:-60
Episode:7 Score:-46
Episode:8 Score:-22
Episode:9 Score:-60
Episode:10 Score:22


In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam

In [ ]:
states = env.observation_space.shape
actions = env.action_space.n

In [ ]:
actions

3

In [ ]:
def build_model(states, actions):
    model = Sequential()    
    model.add(Dense(24, activation='relu', input_shape=states))
    model.add(Dense(24, activation='relu'))
    model.add(Dense(actions, activation='linear'))
    return model

In [ ]:
model = build_model(states, actions)

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 24)                48        
_________________________________________________________________
dense_4 (Dense)              (None, 24)                600       
_________________________________________________________________
dense_5 (Dense)              (None, 3)                 75        
Total params: 723
Trainable params: 723
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from rl.agents import DQNAgent
from rl.policy import BoltzmannQPolicy
from rl.memory import SequentialMemory

In [ ]:
def build_agent(model, actions):
    policy = BoltzmannQPolicy()
    memory = SequentialMemory(limit=50000, window_length=1)
    dqn = DQNAgent(model=model, memory=memory, policy=policy, 
                  nb_actions=actions, nb_steps_warmup=10, target_model_update=1e-2)
    return dqn

In [ ]:
dqn = build_agent(model, actions)
dqn.compile(Adam(learning_rate=1e-3), metrics=['mae'])
dqn.fit(env, nb_steps=50000, visualize=False, verbose=1)

Training for 50000 steps ...
Interval 1 (0 steps performed)
    1/10000 [..............................] - ETA: 5:32 - reward: 1.0000

C:\Users\mink\anaconda3\envs\openai\lib\site-packages\keras\engine\training.py:2470: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '
C:\Users\mink\anaconda3\envs\openai\lib\site-packages\rl\memory.py:37: UserWarning: Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!
  warnings.warn('Not enough entries to sample without replacement. Consider increasing your warm-up phase to avoid oversampling!')


10000/10000 [==============================] - 53s 5ms/step - reward: 0.7920
166 episodes - episode_reward: 47.530 [-60.000, 60.000] - loss: 6.300 - mae: 16.624 - mean_q: 24.965

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 53s 5ms/step - reward: 0.8532
167 episodes - episode_reward: 51.150 [30.000, 60.000] - loss: 13.590 - mae: 27.390 - mean_q: 41.272

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 55s 5ms/step - reward: 0.8736
167 episodes - episode_reward: 52.431 [36.000, 60.000] - loss: 13.684 - mae: 27.458 - mean_q: 41.379

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 57s 6ms/step - reward: 0.8814
166 episodes - episode_reward: 52.952 [34.000, 60.000] - loss: 14.128 - mae: 27.794 - mean_q: 41.864

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 59s 6ms/step - reward: 0.8598
done, took 276.866 seconds


In [ ]:
scores = dqn.test(env, nb_episodes=100, visualize=False)
print(np.mean(scores.history['episode_reward']))

Testing for 100 episodes ...
Episode 1: reward: 60.000, steps: 60
Episode 2: reward: 60.000, steps: 60
Episode 3: reward: 54.000, steps: 60
Episode 4: reward: 54.000, steps: 60
Episode 5: reward: 58.000, steps: 60
Episode 6: reward: 54.000, steps: 60
Episode 7: reward: 60.000, steps: 60
Episode 8: reward: 60.000, steps: 60
Episode 9: reward: 54.000, steps: 60
Episode 10: reward: 60.000, steps: 60
Episode 11: reward: 60.000, steps: 60
Episode 12: reward: 60.000, steps: 60
Episode 13: reward: 56.000, steps: 60
Episode 14: reward: 60.000, steps: 60
Episode 15: reward: 56.000, steps: 60
Episode 16: reward: 60.000, steps: 60
Episode 17: reward: 54.000, steps: 60
Episode 18: reward: 54.000, steps: 60
Episode 19: reward: 58.000, steps: 60
Episode 20: reward: 54.000, steps: 60
Episode 21: reward: 58.000, steps: 60
Episode 22: reward: 60.000, steps: 60
Episode 23: reward: 60.000, steps: 60
Episode 24: reward: 58.000, steps: 60
Episode 25: reward: 60.000, steps: 60
Episode 26: reward: 54.000, st

In [ ]:
_ = dqn.test(env, nb_episodes=15, visualize=True)

Testing for 15 episodes ...


TypeError: render() got an unexpected keyword argument 'mode'